In [ ]:
%%html
<!-- Improve the styling of the Notebook. -->
<link href="https://fonts.googleapis.com/css2?family=Source+Code+Pro&family=Source+Sans+3&family=Source+Serif+4:opsz@8..60&display=swap" rel="stylesheet">
<style>
   div.jp-MarkdownOutput p { font-family: 'Source Serif 4', serif; width: 50em; }
   div.jp-MarkdownOutput h1,h2,h3,h4,h5,h6 { font-family: 'Source Sans 3', sans-serif; }
   div.cm-line { font-family: 'Source Code Pro', monospace; }
</style>

In [ ]:
import hail as hl
hl.init()  # Not necessary, but sometimes you need to configure Hail by passing arguments to hl.init

# Importing a TSV File as a Hail Table

[`hl.import_table`](https://hail.is/docs/0.2/methods/impex.html#hail.methods.import_table), by default, imports tab-separated files, but supports many kinds of delimiters. Hail can import many kinds of files, such as VCF, PLINK, UCSC BED, BGEN, and GEN, see the [Import section](https://hail.is/docs/0.2/methods/impex.html#import) of the docs for details.

In [ ]:
ht = hl.import_table('data/sample_table.tsv', impute=True, min_partitions=2)
ht

The printed form of a table is the inscrutable `<hail.table.Table ...>` because Hail has not yet run anything. The table is just a recipe with one step: import a TSV. We must explicitly request that the recipe is executed with an _action_. We can use the action [`Table.show`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.show) (you can click on that) to see the first few rows:

In [ ]:
ht.show(n=3)

# Describing a Table

We can also use [`Table.describe`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.describe), which is not an action. It lists all the fields the recipe will produce without executing the recipe.

In [ ]:
ht.describe()

Tables can have "keys". If a table has a key, then Hail ensures the table is sorted by its key. Keys are important for combining two tables or combining a table and a matrix table.

# Filtering to Certain Rows

[`Table.filter`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.filter) creates a new recipe that both imports the table _and_ keeps only certain rows:

In [ ]:
ht.filter(ht.age > 30).show(n=3)

Notice that the above command did not modify `ht`. Run `ht.show()` again to verify that:

In [ ]:
ht.show(n=3)

# Head and Tail of the Dataset

[`Table.head`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.head) and [`Table.tail`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.tail) filter the dataset to the first few or last few rows.

In [ ]:
ht.head(5).show()

In [ ]:
ht.tail(5).show()

# Adding New Fields with Annotate

Usually we build up one big recipe by repeatedly mutating the same variable. Let's do that and add a new field using [`Table.annotate`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.annotate):

In [ ]:
ht = ht.annotate(is_twenty_something = hl.all(ht.age >= 20, ht.age < 30))

In [ ]:
ht.show(n=3)

We can also convert the freckles field into a Boolean field with [`hl.case`](https://hail.is/docs/0.2/functions/core.html#hail.expr.functions.case) which is one of the many [core language functions](https://hail.is/docs/0.2/functions/core.html#hail.expr.functions.case) in Hail's standard library.

In [ ]:
ht = ht.annotate(has_freckles = (
    hl.case()
    .when(ht.freckles == "Yes", True)
    .when(ht.freckles == "No", False)
    .or_error(hl.format("Expected \"Yes\" or \"No\" for the field \"freckles\" but found: %s", ht.freckles))
))

In [ ]:
ht.show()

There are two ways to remove the old `freckles` field: [`Table.select`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.select) and [`Table.drop`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.drop):

In [ ]:
ht.select('name', 'age', 'is_twenty_something', 'has_freckles').show(n=3)

In [ ]:
ht.drop('freckles').show(n=3)

# Aggregating a Table to a Single Python Value

Another "action" we can use to execute a Hail table's recipe is [`Table.aggregate`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.aggregate). Let's use the [`hl.agg.mean`](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.mean) aggregator from the [`hl.agg`](https://hail.is/docs/0.2/aggregators.html) module.

In [ ]:
ht.aggregate(hl.agg.mean(ht.age))

Each time we execute an action, the entire table recipe is executed from the beginning. For example, consider how long it takes to execute four aggregations:

In [ ]:
%%time
mean_age = ht.aggregate(hl.agg.mean(ht.age))
sum_age = ht.aggregate(hl.agg.sum(ht.age))
max_age = ht.aggregate(hl.agg.max(ht.age))
min_age = ht.aggregate(hl.agg.min(ht.age))

(mean_age, sum_age, max_age, min_age)

Instead of executing the table's recipe four times, once for each aggregator, we can execute the recipe once computing the four aggregators in parallel:

In [ ]:
%%time
mean_age, sum_age, max_age, min_age = ht.aggregate(
    (
        hl.agg.mean(ht.age),
        hl.agg.sum(ht.age),
        hl.agg.max(ht.age),
        hl.agg.min(ht.age),
    )
)

(mean_age, sum_age, max_age, min_age)

# Aggregating within Groups of Rows to Produce a New Table.

Instead of aggregating over the entire table to produce just one value, we can combine groups of rows into new rows by aggregating over each group separately. We use [`Table.group_by`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.group_by) with [`hl.agg.filter`](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.filter), [`hl.agg.count`](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.count), and [`hl.agg.count_where`](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.count_where).

In [ ]:
ht.group_by(
    ht.age
).aggregate(
    count_having_freckles = hl.agg.filter(ht.freckles == "Yes", hl.agg.count()),
    count_not_having_freckles = hl.agg.filter(ht.freckles == "No", hl.agg.count()), 
    count_names_starting_with_A = hl.agg.count_where(ht.name[0] == "A"),
)

Oops! We forgot to use an action, like [`Table.show`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.show), so nothing happened! Let's try again:

In [ ]:
ht2 = ht.group_by(
    ht.age
).aggregate(
    count_having_freckles = hl.agg.filter(ht.freckles == "Yes", hl.agg.count()),
    count_not_having_freckles = hl.agg.filter(ht.freckles == "No", hl.agg.count()), 
    count_names_starting_with_A = hl.agg.count_where(ht.name[0] == "A"),
)
ht2.show(n=3)

Notice that we used a new variable name, `ht2`, so that we can still access the old table, `ht` containing all the individual people.

In [ ]:
ht.show(n=3)

# Plotting Tables

# Writing and Reading Tables in Hail Native Format

Hail has a partitioned, indexed, binary file format for quickly reading and writing datasets. [`Table.write`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.write) is the action which writes a table in Hail native format. We use the ".ht" file extension by convention.

In [ ]:
ht.write('output/sample_table.ht')

Writing a Hail table executes the recipe once and saves the results in a file for future use. We recommend writing after importing or after executing computationally intensive pipelines. [`hl.read_table`](https://hail.is/docs/0.2/methods/impex.html#hail.methods.read_table) reads a table in Hail native format. Most operations are faster when starting from a Hail native format table.

In [ ]:
ht = hl.read_table('output/sample_table.ht')

# Exporting a Table to a File

Hail tables support export to many file formats including TSV and CSV.

In [ ]:
ht.export('output/sample_table.tsv')
ht.export('output/sample_table.csv', delimiter=',')
ht.export('output/sample_table.@sv', delimiter='@')

In [ ]:
!head output/sample_table.tsv

In [ ]:
!head output/sample_table.csv

In [ ]:
!head output/sample_table.@sv

We did not compress the outputs for ease of viewing. Exporting large tables uncompressed is almost always a mistake. Hail detects the ".bgz" extension and compresses the output using block GZIP. This is almost always faster than exporting an uncompressed text file.

In [ ]:
ht.export('output/sample_table.tsv.bgz')

The `INFO` output mentions a "merge time". This is a slow, serial operation in which Hail concatenates the partitioned dataset into a single file. Whenever possible, you should use partitioned text files. [`Table.export`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.export) exports a folder of partitions when `parallel` is set to `header_per_shard` or `separate_header`.

In [ ]:
ht.export('output/sample_table_partitions_header_per_shard.tsv/', parallel='header_per_shard')

In [ ]:
!head output/sample_table_partitions_header_per_shard.tsv/*

In [ ]:
ht.export('output/sample_table_partitions_separate_header.tsv/', parallel='separate_header')

In [ ]:
!head output/sample_table_partitions_separate_header.tsv/*

# Collecting a Table to a List or Pandas DataFrame

[`Table.collect`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.collect) collects the distributed & partitioned values of a table into a Python list. This will, obviously, run out of memory if the table is large.

In [ ]:
ht.collect()

[`Table.to_pandas`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.to_pandas) collects the values into a Pandas DataFrame. As above, large tables will exceed the memory available on your laptop.

In [ ]:
ht.to_pandas()